In [1]:
import pandas as pd
import numpy as np

from src.preprocess_data import preprocess_dataset, pca_dataset

# Class distribution

Make windows of size 30sec like in original paper. Let them slide 10sec

In [4]:
dfs = preprocess_dataset("condensed", 3000, 1000)

Finished p1_d1. Took 0 minutes, 22 seconds
Finished p1_d2. Took 0 minutes, 33 seconds
Finished p2_d1. Took 0 minutes, 36 seconds
Finished p2_d2. Took 0 minutes, 36 seconds
Finished p3_d1. Took 0 minutes, 46 seconds
Finished p3_d2. Took 0 minutes, 20 seconds
Finished p4_d1. Took 0 minutes, 26 seconds
Finished p4_d2. Took 0 minutes, 26 seconds
Finished p5_d1. Took 0 minutes, 17 seconds
Finished p5_d2. Took 0 minutes, 15 seconds
Finished p6_d1. Took 0 minutes, 19 seconds
Finished p6_d2. Took 0 minutes, 18 seconds
Finished p7_d1. Took 0 minutes, 16 seconds
Finished p7_d2. Took 0 minutes, 16 seconds
Finished p8_d1. Took 0 minutes, 21 seconds
Finished p8_d2. Took 0 minutes, 19 seconds
Finished p9_d1. Took 0 minutes, 29 seconds
Finished p9_d2. Took 0 minutes, 14 seconds
Finished p10_d1. Took 0 minutes, 18 seconds
Finished p10_d2. Took 0 minutes, 31 seconds


In [5]:
cldtr = pd.DataFrame(columns=["person", "Not Reading", "English", "Japanese horizontal", "Japanese vertical"])

for person in range(1,11,1):
    d = np.array([0,0,0,0])
    for day in range(1,3,1):
        key = "p{}_d{}".format(person,day)
        d  += (dfs[key].value_counts(subset="label", sort = False).transpose().to_numpy())
    df = pd.DataFrame({"person" : "person {}".format(person),  "Not Reading" : [d[0]], "English" : [d[1]], "Japanese horizontal" : [d[2]], "Japanese vertical": [d[3]] })
    cldtr = pd.concat([cldtr, df], axis = 0)

total = cldtr.sum().transpose().to_numpy()[1:]
df = pd.DataFrame({"person" : "Sum",  "Not Reading" : [total[0]], "English" : [total[1]], "Japanese horizontal" : [total[2]], "Japanese vertical": [total[3]] })
cldtr = pd.concat([cldtr, df], axis = 0, ignore_index=True)  


print(cldtr)

       person Not Reading English Japanese horizontal Japanese vertical
0    person 1        5641     926                1014               987
1    person 2        5673    1022                 871              1061
2    person 3        5525     816                1309               827
3    person 4        6394     771                 759               730
4    person 5        4558     827                 831              1200
5    person 6        4732    1188                 882               875
6    person 7        3372     957                1085               843
7    person 8        6204     706                 732               843
8    person 9        4699     711                 806              1044
9   person 10        6020     666                 959               847
10        Sum       52818    8590                9248              9257
